In [14]:
import dis
from malthusia.engine.container.instrument import Instrument
import pickle
from pympler import asizeof

In [209]:
s = """
l = str(range(1000))
if type(str) == type and str.__module__ == "builtins":
    print("hi")
l = list()
list.__init__(l, range(10))
l
"""

In [210]:
c = compile(s, "source", "exec")

In [ ]:
# ah, have a function that just takes in as input str and returns an instrumented_str_init_fn

In [211]:
dis.dis(c)

  2           0 LOAD_NAME                0 (str)
              2 LOAD_NAME                1 (range)
              4 LOAD_CONST               0 (1000)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 STORE_NAME               2 (l)

  3          12 LOAD_NAME                3 (type)
             14 LOAD_NAME                0 (str)
             16 CALL_FUNCTION            1
             18 LOAD_NAME                3 (type)
             20 COMPARE_OP               2 (==)
             22 POP_JUMP_IF_FALSE       42
             24 LOAD_NAME                0 (str)
             26 LOAD_ATTR                4 (__module__)
             28 LOAD_CONST               1 ('builtins')
             30 COMPARE_OP               2 (==)
             32 POP_JUMP_IF_FALSE       42

  4          34 LOAD_NAME                5 (print)
             36 LOAD_CONST               2 ('hi')
             38 CALL_FUNCTION            1
             40 POP_TOP

  5     >> 

In [177]:
s = """
class Myerr:
    def __init__(self):
        self.x = "x"
def x():
    print("from x")
i = 0
class X:
    x = [1]
class T(Enum):
    t = 1
    def __init__(self):
        self.s = "omg"
x2x = X()
x2x.xx = "uh"
xx = X()
xx.y = "hej"
xx.xx = xx
def f():
    global i
    X.x.extend([1,2,3,4,5])
    xx.y += "hej"
    x2x.xx = xx.y
    j = 0
    print(j)
    print(i)
    print(Myerr())
    print(X.x)
    i += 1
    x()

"""

In [ ]:
class X(Enum):

In [179]:
from enum import Enum
class X(Enum):
    RED = 1

In [187]:
type(get_dict_repr(X)['_member_type_']['__subclasshook__'])

classmethod_descriptor

In [178]:
d = {}
glbals = {"__builtins__": {"print": print, "frozenset": frozenset, "__build_class__": __build_class__, "__name__": "x", "__import__": __import__}}
exec(s, glbals, d)
glbals.update(d)

NameError: name 'Enum' is not defined

In [171]:
glbals

{'__builtins__': {'print': <function print>,
  'frozenset': frozenset,
  '__build_class__': <function __build_class__>,
  '__name__': 'x',
  '__import__': <function __import__>},
 'i': 0,
 'Myerr': x.Myerr,
 'x': <function x()>,
 'X': x.X,
 'T': x.T,
 'x2x': <x.X at 0x105ad9130>,
 'xx': <x.X at 0x105ad99d0>,
 'f': <function f()>}

In [172]:
glbals['xx'].__dict__

{'y': 'hej', 'xx': <x.X at 0x105ad99d0>}

In [173]:
def get_dict_repr(d, objs=None):
    if objs is None:
        objs = []
    for dd in objs:
        if d is dd:
            return 0 # dont need to count twice
    objs.append(d)
    if hasattr(d, "__dict__"):
        return get_dict_repr(d.__dict__, objs=objs)
    if hasattr(d, "items"):
        nd = {}
        for k, v in d.items():
            nd[k] = get_dict_repr(v, objs=objs)
        return nd
    if isinstance(d, str):
        return d
    if hasattr(d, "__iter__"):
        nd = []
        for v in d:
            nd.append(get_dict_repr(v, objs=objs))
        return nd
    if type(d).__name__ in {"builtin_function_or_method", "getset_descriptor"}:
        return 1
    return d

In [175]:
type(get_dict_repr(glbals)['__builtins__']['frozenset']['copy'])

method_descriptor

In [176]:
get_dict_repr(glbals)

{'__builtins__': {'print': 1,
  'frozenset': {'__repr__': <slot wrapper '__repr__' of 'frozenset' objects>,
   '__hash__': <slot wrapper '__hash__' of 'frozenset' objects>,
   '__getattribute__': <slot wrapper '__getattribute__' of 'frozenset' objects>,
   '__lt__': <slot wrapper '__lt__' of 'frozenset' objects>,
   '__le__': <slot wrapper '__le__' of 'frozenset' objects>,
   '__eq__': <slot wrapper '__eq__' of 'frozenset' objects>,
   '__ne__': <slot wrapper '__ne__' of 'frozenset' objects>,
   '__gt__': <slot wrapper '__gt__' of 'frozenset' objects>,
   '__ge__': <slot wrapper '__ge__' of 'frozenset' objects>,
   '__iter__': <slot wrapper '__iter__' of 'frozenset' objects>,
   '__sub__': <slot wrapper '__sub__' of 'frozenset' objects>,
   '__rsub__': <slot wrapper '__rsub__' of 'frozenset' objects>,
   '__and__': <slot wrapper '__and__' of 'frozenset' objects>,
   '__rand__': <slot wrapper '__rand__' of 'frozenset' objects>,
   '__xor__': <slot wrapper '__xor__' of 'frozenset' object

In [149]:
class X:
    def __init__(self):
        self.x = 1

In [151]:
X().x()

TypeError: 'int' object is not callable

In [147]:
marshal.dumps(get_dict_repr(glbals))

b'{\xda\x0c__builtins__{\xda\x05print\xe9\x01\x00\x00\x00\xda\x0f__build_class__r\x02\x00\x00\x00\xda\x08__name__\xda\x01x\xda\n__import__r\x02\x00\x00\x000\xda\x01i\xe9*\x00\x00\x00\xda\x05Myerr{\xda\n__module__\xe9\x00\x00\x00\x00\xda\x08__init__{0\xda\x08__dict__r\x02\x00\x00\x00\xda\x0b__weakref__r\x02\x00\x00\x00\xda\x07__doc__N0r\x05\x00\x00\x00{0\xda\x01X{r\n\x00\x00\x00r\x0b\x00\x00\x00r\x05\x00\x00\x00[\xd3\x00\x00\x00r\x02\x00\x00\x00r\x0b\x00\x00\x00\xe9\x02\x00\x00\x00\xe9\x03\x00\x00\x00\xe9\x04\x00\x00\x00\xe9\x05\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00\x00r\x0b\x00\x00

In [146]:
pickle.dumps(get_dict_repr(glbals))

b'\x80\x04\x95\x1a\x03\x00\x00\x00\x00\x00\x00}\x94(\x8c\x0c__builtins__\x94}\x94(\x8c\x05print\x94K\x01\x8c\x0f__build_class__\x94K\x01\x8c\x08__name__\x94\x8c\x01x\x94\x8c\n__import__\x94K\x01u\x8c\x01i\x94K*\x8c\x05Myerr\x94}\x94(\x8c\n__module__\x94K\x00\x8c\x08__init__\x94}\x94\x8c\x08__dict__\x94K\x01\x8c\x0b__weakref__\x94K\x01\x8c\x07__doc__\x94Nuh\x06}\x94\x8c\x01X\x94}\x94(h\x0bK\x00h\x06]\x94(K\x01K\x00K\x02K\x03K\x04K\x05K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x

In [93]:
get_dict_repr(glbals)

{'__builtins__': {'print': 1,
  '__build_class__': 1,
  '__name__': 'x',
  '__import__': 1},
 'i': 0,
 'Myerr': {'__module__': 0,
  '__init__': {},
  '__dict__': 1,
  '__weakref__': 1,
  '__doc__': None},
 'x': {},
 'X': {'__module__': 0,
  'x': [1],
  '__dict__': 1,
  '__weakref__': 1,
  '__doc__': 0},
 'xx': {'y': 'hej', 'xx': 0},
 'f': {}}

In [116]:
exec(glbals['f'].__code__, glbals, {})

0
15
[1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5]
from x


In [285]:
glbals

{'__builtins__': {'print': <function print>,
  '__build_class__': <function __build_class__>,
  '__name__': 'x'},
 'i': 0,
 'Myerr': x.Myerr,
 'x': <function x()>,
 'f': <function f()>}

In [231]:
d

{'Myerr': x.Myerr, 'x': <function x()>, 'f': <function f()>}

In [145]:
exec(d['f'].__code__, glbals, {})

0
41
[1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5]
from x


In [234]:
import json

In [237]:
import sys

In [239]:
import gc
import sys

def get_obj_size(obj):
    marked = {id(obj)}
    obj_q = [obj]
    sz = 0

    while obj_q:
        sz += sum(map(sys.getsizeof, obj_q))

        # Lookup all the object referred to by the object in obj_q.
        # See: https://docs.python.org/3.7/library/gc.html#gc.get_referents
        all_refr = ((id(o), o) for o in gc.get_referents(*obj_q))

        # Filter object that are already marked.
        # Using dict notation will prevent repeated objects.
        new_refr = {o_id: o for o_id, o in all_refr if o_id not in marked and not isinstance(o, type)}

        # The new obj_q will be the ones that were not marked,
        # and we will update marked with their ids so we will
        # not traverse them again.
        obj_q = new_refr.values()
        marked.update(new_refr.keys())

    return sz


In [246]:
glbals["__builtins__"]["print"] = lambda x: 1

{'__builtins__': {'print': <function __main__.<lambda>(x)>,
  '__build_class__': <function __build_class__>,
  '__name__': 'x'},
 'i': 0,
 'Myerr': x.Myerr,
 'x': <function x()>,
 'f': <function f()>}

In [250]:
glbals

{'__builtins__': {'print': <function __main__.<lambda>(x)>,
  '__build_class__': <function __build_class__>,
  '__name__': 'x'},
 'i': 0,
 'Myerr': x.Myerr,
 'x': <function x()>,
 'f': <function f()>}

In [247]:
get_obj_size(glbals)

29778392

In [240]:
sys.getsizeof(glbals)

360

In [235]:
json.dumps(glbals)

TypeError: Object of type builtin_function_or_method is not JSON serializable

In [233]:
newglbals = glbals
newglbals['__builtins__']["__import__"] = __import__
newglbals["__globals"] = glbals
d = {}
exec(f"import pickle\nx = pickle.dumps(__globals)", newglbals, d)

PicklingError: Can't pickle <class 'x.Myerr'>: import of module 'x' failed

In [221]:
class Hi:
    def __init__(self):
        self.x = "x"

In [222]:
pickle.dumps({"x": Hi})

b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x01x\x94\x8c\x08__main__\x94\x8c\x02Hi\x94\x93\x94s.'

In [6]:
s = """
def turn():
    try:
        n = get_bytecode()
        log(f"bytecode: {n}")
        turn()
    except (RecursionError, MemoryError):
        raise
    except:
        turn()
turn()
"""
c = compile(s, "source", "exec")

In [7]:
ci = Instrument.instrument(c, replace_builtins=False, instrument=False, instrument_binary_multiply=True, reraise_dangerous_exceptions=True)


In [8]:
c.co_names

('turn',)

In [9]:
new_builtins = {"__builtins__": 
                {"__instrumented_index": lambda x, y: 32, 
                 "type": type, "__safe_type__": type, 
                 "print": print, 
                 "__instrument_binary_multiply__": lambda a,b: print(f"instrumenting {a} * {b}"), 
                 "__build_class__": __build_class__, 
                 "__name__": "DANGEROUS_main",
                 "RecursionError": RecursionError,
                 "MemoryError": MemoryError,
                 "__instrument__": lambda: None
                }
               }

In [ ]:
exec(c, new_builtins)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  [Previous line repeated 996 more times]
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.p

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

...

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

    _seen.add(id(exc_value))
RecursionError: maximum recursion depth exceeded while calling a Python object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  [Previous line repeated 996 more times]
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 476, in __init__
    _seen.add(id(exc_value))
RecursionError: maximum recursion depth exceeded while calling a Python object

During handling of the above e

In [21]:
exec(ci, new_builtins)

In [15]:
dis.dis(c)

  2           0 LOAD_CONST               0 (<code object turn at 0x10c23c920, file "source", line 2>)
              2 LOAD_CONST               1 ('turn')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (turn)
              8 LOAD_CONST               2 (None)
             10 RETURN_VALUE

Disassembly of <code object turn at 0x10c23c920, file "source", line 2>:
  3           0 SETUP_FINALLY           30 (to 32)

  4           2 LOAD_GLOBAL              0 (get_bytecode)
              4 CALL_FUNCTION            0
              6 STORE_FAST               0 (n)

  5           8 LOAD_GLOBAL              1 (log)
             10 LOAD_CONST               1 ('bytecode: ')
             12 LOAD_FAST                0 (n)
             14 FORMAT_VALUE             0
             16 BUILD_STRING             2
             18 CALL_FUNCTION            1
             20 POP_TOP

  6          22 LOAD_GLOBAL              2 (turn)
             24 CALL_FUNCTION            

In [9]:
dis.dis(ci)

  2           0 LOAD_CONST               0 (<code object turn at 0x10c1a5920, file "source", line 2>)
              2 LOAD_CONST               1 ('turn')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (turn)
              8 LOAD_CONST               2 (None)
             10 RETURN_VALUE

Disassembly of <code object turn at 0x10c1a5920, file "source", line 2>:
  3           0 SETUP_FINALLY           30 (to 32)

  4           2 LOAD_GLOBAL              0 (get_bytecode)
              4 CALL_FUNCTION            0
              6 STORE_FAST               0 (n)

  5           8 LOAD_GLOBAL              1 (log)
             10 LOAD_CONST               1 ('bytecode: ')
             12 LOAD_FAST                0 (n)
             14 FORMAT_VALUE             0
             16 BUILD_STRING             2
             18 CALL_FUNCTION            1
             20 POP_TOP

  6          22 LOAD_GLOBAL              2 (turn)
             24 CALL_FUNCTION            

In [13]:
s1 = """  2           0 LOAD_GLOBAL              8 (__instrument__)
              2 CALL_FUNCTION            0
              4 POP_TOP
              6 LOAD_CONST               0 ('hej')
              8 LOAD_GLOBAL              8 (__instrument__)
             10 CALL_FUNCTION            0
             12 POP_TOP
             14 STORE_NAME               0 (x)

  3          16 LOAD_GLOBAL              8 (__instrument__)
             18 CALL_FUNCTION            0
             20 POP_TOP
             22 SETUP_FINALLY           48 (to 72)

  4          24 LOAD_GLOBAL              8 (__instrument__)
             26 CALL_FUNCTION            0
             28 POP_TOP
             30 LOAD_CONST               1 (12)
             32 LOAD_GLOBAL              8 (__instrument__)
             34 CALL_FUNCTION            0
             36 POP_TOP
             38 LOAD_CONST               2 (0)
             40 LOAD_GLOBAL              8 (__instrument__)
             42 CALL_FUNCTION            0
             44 POP_TOP
             46 BINARY_TRUE_DIVIDE
             48 LOAD_GLOBAL              8 (__instrument__)
             50 CALL_FUNCTION            0
             52 POP_TOP
             54 STORE_NAME               0 (x)
             56 LOAD_GLOBAL              8 (__instrument__)
             58 CALL_FUNCTION            0
             60 POP_TOP
             62 POP_BLOCK
             64 LOAD_GLOBAL              8 (__instrument__)
             66 CALL_FUNCTION            0
             68 POP_TOP
             70 JUMP_FORWARD           104 (to 176)
        >>   72 DUP_TOP
             74 LOAD_NAME                5 (RecursionError)
             76 LOAD_NAME                6 (MemoryError)
             78 BUILD_TUPLE              2
             80 COMPARE_OP              10 (exception match)
             82 POP_JUMP_IF_FALSE       96
             84 POP_TOP
             86 POP_TOP
             88 POP_TOP
             90 RAISE_VARARGS            0
             92 POP_EXCEPT
             94 JUMP_FORWARD            80 (to 176)

  5     >>   96 LOAD_GLOBAL              8 (__instrument__)
             98 CALL_FUNCTION            0
            100 POP_TOP
            102 POP_TOP
            104 LOAD_GLOBAL              8 (__instrument__)
            106 CALL_FUNCTION            0
            108 POP_TOP
            110 POP_TOP
            112 LOAD_GLOBAL              8 (__instrument__)
            114 CALL_FUNCTION            0
            116 POP_TOP
            118 POP_TOP

  6         120 LOAD_GLOBAL              8 (__instrument__)
            122 CALL_FUNCTION            0
            124 POP_TOP
            126 LOAD_NAME                1 (print)
            128 LOAD_GLOBAL              8 (__instrument__)
            130 CALL_FUNCTION            0
            132 POP_TOP
            134 LOAD_CONST               3 ('ohno')
            136 LOAD_GLOBAL              8 (__instrument__)
            138 CALL_FUNCTION            0
            140 POP_TOP
            142 CALL_FUNCTION            1
            144 LOAD_GLOBAL              8 (__instrument__)
            146 CALL_FUNCTION            0
            148 POP_TOP
            150 POP_TOP
            152 LOAD_GLOBAL              8 (__instrument__)
            154 CALL_FUNCTION            0
            156 POP_TOP
            158 POP_EXCEPT
            160 LOAD_GLOBAL              8 (__instrument__)
            162 CALL_FUNCTION            0
            164 POP_TOP
            166 JUMP_FORWARD             8 (to 176)
            168 LOAD_GLOBAL              8 (__instrument__)
            170 CALL_FUNCTION            0
            172 POP_TOP
            174 END_FINALLY

  7     >>  176 LOAD_GLOBAL              8 (__instrument__)
            178 CALL_FUNCTION            0
            180 POP_TOP
            182 LOAD_CONST               4 (3)
            184 LOAD_GLOBAL              8 (__instrument__)
            186 CALL_FUNCTION            0
            188 POP_TOP
            190 STORE_NAME               0 (x)

  8         192 LOAD_GLOBAL              8 (__instrument__)
            194 CALL_FUNCTION            0
            196 POP_TOP
            198 LOAD_CONST               5 (4)
            200 LOAD_GLOBAL              8 (__instrument__)
            202 CALL_FUNCTION            0
            204 POP_TOP
            206 STORE_NAME               2 (y)

  9         208 LOAD_GLOBAL              8 (__instrument__)
            210 CALL_FUNCTION            0
            212 POP_TOP
            214 LOAD_NAME                0 (x)
            216 LOAD_GLOBAL              8 (__instrument__)
            218 CALL_FUNCTION            0
            220 POP_TOP
            222 LOAD_NAME                2 (y)
            224 LOAD_GLOBAL              8 (__instrument__)
            226 CALL_FUNCTION            0
            228 POP_TOP
            230 BINARY_ADD
            232 LOAD_GLOBAL              8 (__instrument__)
            234 CALL_FUNCTION            0
            236 POP_TOP
            238 STORE_NAME               3 (z)
            240 LOAD_GLOBAL              8 (__instrument__)
            242 CALL_FUNCTION            0
            244 POP_TOP
            246 LOAD_CONST               6 (None)
            248 LOAD_GLOBAL              8 (__instrument__)
            250 CALL_FUNCTION            0
            252 POP_TOP
            254 RETURN_VALUE
"""

In [14]:
s2="""  2           0 LOAD_GLOBAL              7 (__instrument__)
              2 CALL_FUNCTION            0
              4 POP_TOP
              6 LOAD_CONST               0 ('hej')
              8 LOAD_GLOBAL              7 (__instrument__)
             10 CALL_FUNCTION            0
             12 POP_TOP
             14 STORE_NAME               0 (x)

  3          16 LOAD_GLOBAL              7 (__instrument__)
             18 CALL_FUNCTION            0
             20 POP_TOP
             22 SETUP_FINALLY           48 (to 72)

  4          24 LOAD_GLOBAL              7 (__instrument__)
             26 CALL_FUNCTION            0
             28 POP_TOP
             30 LOAD_CONST               1 (12)
             32 LOAD_GLOBAL              7 (__instrument__)
             34 CALL_FUNCTION            0
             36 POP_TOP
             38 LOAD_CONST               2 (0)
             40 LOAD_GLOBAL              7 (__instrument__)
             42 CALL_FUNCTION            0
             44 POP_TOP
             46 BINARY_TRUE_DIVIDE
             48 LOAD_GLOBAL              7 (__instrument__)
             50 CALL_FUNCTION            0
             52 POP_TOP
             54 STORE_NAME               0 (x)
             56 LOAD_GLOBAL              7 (__instrument__)
             58 CALL_FUNCTION            0
             60 POP_TOP
             62 POP_BLOCK
             64 LOAD_GLOBAL              7 (__instrument__)
             66 CALL_FUNCTION            0
             68 POP_TOP
             70 JUMP_FORWARD           176 (to 248)

  5     >>   72 LOAD_GLOBAL              7 (__instrument__)
             74 CALL_FUNCTION            0
             76 POP_TOP
             78 DUP_TOP
             80 LOAD_GLOBAL              7 (__instrument__)
             82 CALL_FUNCTION            0
             84 POP_TOP
             86 LOAD_NAME                1 (RecursionError)
             88 LOAD_GLOBAL              7 (__instrument__)
             90 CALL_FUNCTION            0
             92 POP_TOP
             94 LOAD_NAME                2 (MemoryError)
             96 LOAD_GLOBAL              7 (__instrument__)
             98 CALL_FUNCTION            0
            100 POP_TOP
            102 BUILD_TUPLE              2
            104 LOAD_GLOBAL              7 (__instrument__)
            106 CALL_FUNCTION            0
            108 POP_TOP
            110 COMPARE_OP              10 (exception match)
            112 LOAD_GLOBAL              7 (__instrument__)
            114 CALL_FUNCTION            0
            116 POP_TOP
            118 POP_JUMP_IF_FALSE      168
            120 LOAD_GLOBAL              7 (__instrument__)
            122 CALL_FUNCTION            0
            124 POP_TOP
            126 POP_TOP
            128 LOAD_GLOBAL              7 (__instrument__)
            130 CALL_FUNCTION            0
            132 POP_TOP
            134 POP_TOP
            136 LOAD_GLOBAL              7 (__instrument__)
            138 CALL_FUNCTION            0
            140 POP_TOP
            142 POP_TOP

  6         144 LOAD_GLOBAL              7 (__instrument__)
            146 CALL_FUNCTION            0
            148 POP_TOP
            150 RAISE_VARARGS            0
            152 LOAD_GLOBAL              7 (__instrument__)
            154 CALL_FUNCTION            0
            156 POP_TOP
            158 POP_EXCEPT
            160 LOAD_GLOBAL              7 (__instrument__)
            162 CALL_FUNCTION            0
            164 POP_TOP
            166 JUMP_FORWARD            80 (to 248)

  7     >>  168 LOAD_GLOBAL              7 (__instrument__)
            170 CALL_FUNCTION            0
            172 POP_TOP
            174 POP_TOP
            176 LOAD_GLOBAL              7 (__instrument__)
            178 CALL_FUNCTION            0
            180 POP_TOP
            182 POP_TOP
            184 LOAD_GLOBAL              7 (__instrument__)
            186 CALL_FUNCTION            0
            188 POP_TOP
            190 POP_TOP

  8         192 LOAD_GLOBAL              7 (__instrument__)
            194 CALL_FUNCTION            0
            196 POP_TOP
            198 LOAD_NAME                3 (print)
            200 LOAD_GLOBAL              7 (__instrument__)
            202 CALL_FUNCTION            0
            204 POP_TOP
            206 LOAD_CONST               3 ('ohno')
            208 LOAD_GLOBAL              7 (__instrument__)
            210 CALL_FUNCTION            0
            212 POP_TOP
            214 CALL_FUNCTION            1
            216 LOAD_GLOBAL              7 (__instrument__)
            218 CALL_FUNCTION            0
            220 POP_TOP
            222 POP_TOP
            224 LOAD_GLOBAL              7 (__instrument__)
            226 CALL_FUNCTION            0
            228 POP_TOP
            230 POP_EXCEPT
            232 LOAD_GLOBAL              7 (__instrument__)
            234 CALL_FUNCTION            0
            236 POP_TOP
            238 JUMP_FORWARD             8 (to 248)
            240 LOAD_GLOBAL              7 (__instrument__)
            242 CALL_FUNCTION            0
            244 POP_TOP
            246 END_FINALLY

  9     >>  248 LOAD_GLOBAL              7 (__instrument__)
            250 CALL_FUNCTION            0
            252 POP_TOP
            254 LOAD_CONST               4 (3)
            256 LOAD_GLOBAL              7 (__instrument__)
            258 CALL_FUNCTION            0
            260 POP_TOP
            262 STORE_NAME               0 (x)

 10         264 LOAD_GLOBAL              7 (__instrument__)
            266 CALL_FUNCTION            0
            268 POP_TOP
            270 LOAD_CONST               5 (4)
            272 LOAD_GLOBAL              7 (__instrument__)
            274 CALL_FUNCTION            0
            276 POP_TOP
            278 STORE_NAME               4 (y)

 11         280 LOAD_GLOBAL              7 (__instrument__)
            282 CALL_FUNCTION            0
            284 POP_TOP
            286 LOAD_NAME                0 (x)
            288 LOAD_GLOBAL              7 (__instrument__)
            290 CALL_FUNCTION            0
            292 POP_TOP
            294 LOAD_NAME                4 (y)
            296 LOAD_GLOBAL              7 (__instrument__)
            298 CALL_FUNCTION            0
            300 POP_TOP
            302 BINARY_ADD
            304 LOAD_GLOBAL              7 (__instrument__)
            306 CALL_FUNCTION            0
            308 POP_TOP
            310 STORE_NAME               5 (z)
            312 LOAD_GLOBAL              7 (__instrument__)
            314 CALL_FUNCTION            0
            316 POP_TOP
            318 LOAD_CONST               6 (None)
            320 LOAD_GLOBAL              7 (__instrument__)
            322 CALL_FUNCTION            0
            324 POP_TOP
            326 RETURN_VALUE
"""